<a href="https://colab.research.google.com/github/Rahul21388/Python-Chess-Game/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 600, 600
SQUARE_SIZE = WIDTH // 8
WHITE = (240, 217, 181)
BLACK = (181, 136, 99)

# Chess Unicode symbols
piece_symbols = {
    "wP": "♙", "bP": "♟",
    "wR": "♖", "bR": "♜",
    "wN": "♘", "bN": "♞",
    "wB": "♗", "bB": "♝",
    "wQ": "♕", "bQ": "♛",
    "wK": "♔", "bK": "♚"
}

# Chessboard representation
board = [
    ["bR", "bN", "bB", "bQ", "bK", "bB", "bN", "bR"],
    ["bP"] * 8,
    ["."] * 8,
    ["."] * 8,
    ["."] * 8,
    ["."] * 8,
    ["wP"] * 8,
    ["wR", "wN", "wB", "wQ", "wK", "wB", "wN", "wR"]
]

# Pygame window
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Chess Game")

# ✅ Load a TrueType Font (TTF) for proper Unicode support
try:
    font = pygame.font.Font("FreeSerif.ttf", 50)  # Ensure FreeSerif.ttf is in the same folder
except:
    font = pygame.font.Font(None, 50)  # Fallback if FreeSerif.ttf is missing

selected_piece = None  # Stores the selected piece position
turn = "white"  # White moves first

# Draw the chessboard
def draw_board():
    for row in range(8):
        for col in range(8):
            color = WHITE if (row + col) % 2 == 0 else BLACK
            pygame.draw.rect(win, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

# Draw pieces on the board
def draw_pieces():
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece in piece_symbols:
                text_color = (0, 0, 0) if piece.startswith("b") else (255, 255, 255)  # Black or White pieces
                text = font.render(piece_symbols[piece], True, text_color)
                win.blit(text, (col * SQUARE_SIZE + 15, row * SQUARE_SIZE + 10))

# Convert pixel coordinates to board index
def get_board_pos(x, y):
    return y // SQUARE_SIZE, x // SQUARE_SIZE

# Move validation
def is_valid_move(piece, start, end):
    start_row, start_col = start
    end_row, end_col = end

    if start == end:  # Prevent moving to the same square
        return False

    if board[end_row][end_col] != "." and board[start_row][start_col][0] == board[end_row][end_col][0]:  # Prevent capturing own piece
        return False

    # Pawn movement
    if piece == "wP":
        if start_col == end_col and board[end_row][end_col] == ".":
            if end_row == start_row - 1 or (start_row == 6 and end_row == 4 and board[5][end_col] == "."):
                return True
        elif abs(start_col - end_col) == 1 and end_row == start_row - 1 and board[end_row][end_col].startswith("b"):
            return True
    elif piece == "bP":
        if start_col == end_col and board[end_row][end_col] == ".":
            if end_row == start_row + 1 or (start_row == 1 and end_row == 3 and board[2][end_col] == "."):
                return True
        elif abs(start_col - end_col) == 1 and end_row == start_row + 1 and board[end_row][end_col].startswith("w"):
            return True

    # Rook movement (must check for obstacles)
    if piece in ["wR", "bR"]:
        if start_row == end_row:  # Moving horizontally
            step = 1 if start_col < end_col else -1
            for col in range(start_col + step, end_col, step):
                if board[start_row][col] != ".":
                    return False  # Obstacle in the way
            return True
        elif start_col == end_col:  # Moving vertically
            step = 1 if start_row < end_row else -1
            for row in range(start_row + step, end_row, step):
                if board[row][start_col] != ".":
                    return False  # Obstacle in the way
            return True

    # Bishop movement (must check for obstacles)
    if piece in ["wB", "bB"]:
        if abs(start_row - end_row) == abs(start_col - end_col):
            row_step = 1 if start_row < end_row else -1
            col_step = 1 if start_col < end_col else -1
            row, col = start_row + row_step, start_col + col_step
            while row != end_row and col != end_col:
                if board[row][col] != ".":
                    return False  # Obstacle in the way
                row += row_step
                col += col_step
            return True

    # Queen movement (combines Rook + Bishop logic)
    if piece in ["wQ", "bQ"]:
        if start_row == end_row or start_col == end_col:  # Rook-like movement
            return is_valid_move("wR", start, end)  # Use the Rook logic
        elif abs(start_row - end_row) == abs(start_col - end_col):  # Bishop-like movement
            return is_valid_move("wB", start, end)  # Use the Bishop logic

    # Knight movement (does NOT need path checking)
    if piece in ["wN", "bN"]:
        if (abs(start_row - end_row), abs(start_col - end_col)) in [(2, 1), (1, 2)]:
            return True

    # King movement
    if piece in ["wK", "bK"]:
        if abs(start_row - end_row) <= 1 and abs(start_col - end_col) <= 1:
            return True

    return False  # If no valid move was found


# Handle piece selection and movement
def handle_click(pos):
    global selected_piece, turn

    row, col = get_board_pos(*pos)
    print(f"Clicked on: {row}, {col}")

    if selected_piece:
        start_row, start_col = selected_piece
        piece = board[start_row][start_col]

        print(f"Trying to move {piece} from {start_row},{start_col} to {row},{col}")

        if (turn == "white" and piece.startswith("b")) or (turn == "black" and piece.startswith("w")):
            print("Not your turn!")
            selected_piece = None
            return

        if is_valid_move(piece, (start_row, start_col), (row, col)):
            board[row][col] = piece
            board[start_row][start_col] = "."
            turn = "black" if turn == "white" else "white"
            print("Move successful!")
        else:
            print("Invalid move!")

        selected_piece = None
    else:
        if board[row][col] in piece_symbols:
            selected_piece = (row, col)
            print(f"Selected {board[row][col]} at {row}, {col}")

# Game loop
running = True
while running:
    draw_board()
    draw_pieces()
    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            handle_click(event.pos)

pygame.quit()
sys.exit()


KeyboardInterrupt: 